### Content

In [1]:
# Widgets:
import ipywidgets as widgets

# CSV/DataFrame:
import pandas as pd
import numpy as np
from io import StringIO

# Drawing:
import matplotlib.pyplot as plt
%matplotlib inline

### Widgets

### Tabs

In [23]:
tab = widgets.Tab()
placeholder = widgets.Label()
tab.children = [placeholder,placeholder,placeholder]
tab.set_title(0, "Upload")
tab.set_title(1, "Describer")
tab.set_title(2, "Plotter")
tab

# Fileupload Widget

In [24]:
up = widgets.FileUpload(accept="", multiple=False)
up

FileUpload(value=(), description='Upload')

### Output Widget 

In [25]:
out = widgets.Output(layout={'border': '1px solid black'})
tab.children = [up,out,out]
tab

### SelectMultiple Widget

In [26]:
eraser = widgets.SelectMultiple(
    options=['tab','"'],
    volue=['tab'],
    #rows=10,
    descripton='Eraser: ',
    disable=False
)
eraser

SelectMultiple(options=('tab', '"'), value=())

### RadioButtons Widget

In [27]:
delim = widgets.RadioButtons(
    options=[';',',',' '],
    description='Separator: ',
    disabled=False
)
delim

RadioButtons(description='Separator: ', options=(';', ',', ' '), value=';')

### IntSlider

In [7]:
rows = widgets.IntSlider(
    value=0,
    step=1,
    description='# of lines:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
rows

IntSlider(value=0, continuous_update=False, description='# of lines:')

### Accordion Widget

In [28]:
accordion = widgets.Accordion()
accordion.children=[
    up,
    delim,
    rows]
accordion.set_title(0, 'File Selection')
accordion.set_title(1, 'Delimiter')
accordion.set_title(2, 'Skip Rows')
accordion

Accordion(children=(FileUpload(value=(), description='Upload'), RadioButtons(description='Separator: ', option…

### Button Widget

In [9]:
button_upload = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='warning',
    tooltip='Click to Upload',
    icon='check'
)
button_upload

Button(button_style='warning', description='Upload', icon='check', style=ButtonStyle(), tooltip='Click to Uplo…

In [10]:
button_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search'
)
button_preview

Button(button_style='info', description='Preview', icon='search', style=ButtonStyle(), tooltip='Click to Previ…

In [11]:
button_plot = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='danger',
    tooltip='Click to Plot',
    icon='pencil'
)
button_plot

Button(button_style='danger', description='Plot', icon='pencil', style=ButtonStyle(), tooltip='Click to Plot')

### Groupping - HBox/VBox

In [31]:
vb = widgets.VBox([delim, eraser])
vb

In [30]:
hb = widgets.HBox([delim, eraser])
hb

In [32]:
accordion.children=[
    up,
    widgets.VBox([delim, eraser]),
    rows]

accordion_box = widgets.VBox([
    accordion,
    widgets.HBox([button_preview, button_upload]),
    out
])
accordion_box

### ToggleButtons Widget

In [33]:
toggle = widgets.ToggleButtons(
    options=['Preview ' , 'Info ' , 'Stats '],
    description='Options',
    disabled=False,
    button_style='warning',
    icons=['search', 'info', 'tachometer']
)
#toggle.observe(desc_clicked, 'value'))
toggle

ToggleButtons(button_style='warning', description='Options', icons=('search', 'info', 'tachometer'), options=(…

### Dropdown Widget

In [34]:
x_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='X-Axis:',
    disabled=False,
)

y_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='X-Axis:',
    disabled=False, 
)

graph_type = widgets.Dropdown(
    options=['Bar Chart', 'Line Chart'],
    value='Bar Chart',
    description='Chart Type:',
    disabled=False,  
)
graph_type

Dropdown(description='Chart Type:', options=('Bar Chart', 'Line Chart'), value='Bar Chart')

### ColorPicker Widget

In [36]:
color_picker = widgets.ColorPicker(
    concise=False,
    description='Color Picker:',
    value='lightblue',
    disabled=False
)
color_picker

ColorPicker(value='lightblue', description='Color Picker:')

In [37]:
children = [
    accordion_box,
    widgets.VBox([toggle, out]),
    widgets.VBox([
        widgets.HBox([graph_type, color_picker]),
        widgets.HBox([x_axis, y_axis]),
        button_plot,
        out
])]
tab.children = children
tab

### Value Import

In [38]:
def content_parser():
    if  up.value == {}:
        with out:
            print('No CVS loaded')
    else:
        typ, content = "", ""
        up_value = up.value
        for i in up_value.keys():
            typ = up_value[i]["metadata"]["type"]
            
            if typ == "text/csv":
                content = up_value[i]["content"]
                content_str = str(content, 'utf-8')

                if eraser.value != {}:
                    for val in eraser.value:
                        if val == "tab":
                           content_str = content_str.replace("\t","")
                        else:
                           content_str = content_str.replace("val", "")
        if content_str !="":
            str_io = StringIO(content_str)
            return str_io
def df_converter():
    content = content_parser()
    if content is not None:
        df = pd.read_csv(content, sep=delim.value, index_col=False, skiprows=rows.value)
        return df
    else:
        return None

In [41]:
tab

In [43]:
df_converter()

AttributeError: 'tuple' object has no attribute 'keys'

### Button Action

### Preview Button

In [49]:
def preview():
    df = df_converter()
    with out:
        out.clear_output()
        print('\n ------Now this is how your DF looks like:-------\n')
        if df is not None:
            print(df.head(10))
        else:
            print('Configuration is wrong/missing...')
            

In [46]:
def preview_clicked(b):
    preview()

button_preview.on_click(preview_clicked)

### Upload Button

In [47]:
def upload():
    df = df_converter()
    with out:
        out.clear_output()
        print('\n ------Your uploaded DF  looks like:-----\n')
        if df is not None:
            print(df)
            x_axis.options = df.columns
            y_axis.options = df.columns
        else:
            print('Configuration is wrong/missing...')

In [48]:
def upload_clicked(b):
    upload()

button_upload.on_click(upload_clicked)

### Description ToggleButton

In [51]:
def desc():
    info_level = toggle.value
    if info_level != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n -------Your {} looks like:--------\n'.format(info_level))
            if df is not None:
                if info_level == 'Info  ':
                    print(df.info(verbose=True))
                elif info_level == 'Stats   ':
                    print(df.describe())
                elif info_level == 'Preview  ':
                    print(df.head(5))
                else:
                    print('Configuration is wrong/missing...')

In [52]:
def desc_clicked(b):
    desc()

toggle.observe(desc_clicked, "value")

### Plot Button

In [53]:
def plot():
    graph = graph_type.value
    if graph != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n -------Your {} looks like:-----  \n'.format(graph))
            if (df is not None):
                #df = df.head(5)
                height = df[y_axis.value]
                bars = df[x_axis.value]
                y_pos = np.arange(len(height))
                plt.figure(figsize=(10,4))
                if graph == 'Bar Chart':


                    # Create bars
                    plt.bar(
                        y_pos,
                        height,
                        color=color_picker.value)

                    # Create names on the x-axis
                    plt.xticks(y_pos, bars)
                elif graph == 'Line Chart':
                    plt.plot(
                        bars,
                        height,
                        color=color_picker.value,
                        marker='o',
                        linestyle='solid'
                    )
                    plt.xticks(bars)

                    # Show graphic
                    plt.show()

In [54]:
def plotter_clicked(b):
    plot()

button_plot.on_click(plotter_clicked)

In [56]:
tab